In [ ]:
import cv2
import mediapipe as mp
import urllib.request
import numpy as np
import pickle
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import animation
import PyQt5
from PIL import Image
from IPython.display import Video
import nb_helpers
import os
import csv

mp.drawings = mp.solutions.drawing_utils

Delete frames

In [ ]:
import os
import shutil

def delete_video_folders(base_path='data/users'):
    """Elimina tutte le cartelle che contengono i frame, comprese le cartelle con il nome del video."""
    for user in os.listdir(base_path):
        videos_path = os.path.join(base_path, user, 'videos')

        if not os.path.isdir(videos_path):  # Controlla che "videos" sia una cartella
            continue  

        for video_folder in os.listdir(videos_path):
            video_folder_path = os.path.join(videos_path, video_folder)

            # Se è una cartella (quindi una di quelle create per i frame), la eliminiamo
            if os.path.isdir(video_folder_path):
                shutil.rmtree(video_folder_path)
                print(f"Cartella eliminata: {video_folder_path}")

delete_video_folders()


Extract frame from video

In [12]:
import os
import cv2

base_path = 'data/users/d1cb40d5abd6da294762d26746e64ab7e10c379b'

for user in os.listdir(base_path):
    videos_path = os.path.join(base_path, 'videos')

    if not os.path.isdir(videos_path):  # Controlla che "videos" sia una cartella
        print("No folder")
        continue

        # take only the first video
    video = os.listdir(videos_path)[1]
    video_path = os.path.join(videos_path, video)

    if os.path.isfile(video_path) and video.endswith('.mp4'):  # Controlla che sia un file video
        video_name = video.rsplit('.mp4', 1)[0]  # Rimuove .mp4 dal nome
        frames_folder = os.path.join(base_path, user, 'frames', video_name)

        os.makedirs(frames_folder, exist_ok=True)  # Crea la cartella per i frame all'interno di "frames"

        # Apri il video con OpenCV
        cap = cv2.VideoCapture(video_path)
        frame_count = 0

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break  # Esce quando il video finisce

            frame_filename = os.path.join(frames_folder, f'frame_{frame_count:05d}.jpg')
            cv2.imwrite(frame_filename, frame)  # Salva il frame
            frame_count += 1

        cap.release()  # Chiude il file video

        print(f"Salvati {frame_count} frame in {frames_folder}")

    else:
        print(video_path)


Salvati 122 frame in data/users/d1cb40d5abd6da294762d26746e64ab7e10c379b/videos/frames/d1cb40d5abd6da294762d26746e64ab7e10c379b_inquietanti_aa3d8e9f-7c1a-4a0e-be47-4a9766592c15.jpg
Salvati 122 frame in data/users/d1cb40d5abd6da294762d26746e64ab7e10c379b/tensors/frames/d1cb40d5abd6da294762d26746e64ab7e10c379b_inquietanti_aa3d8e9f-7c1a-4a0e-be47-4a9766592c15.jpg


Landmarks and RGB tensor

In [ ]:
import os
import cv2
import numpy as np
import mediapipe as mp
import matplotlib.pyplot as plt

# Inizializza MediaPipe
mp_face_mesh = mp.solutions.face_mesh
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

def process_frames_and_plot():
    files = sorted(os.listdir('data/frames'))
    num_frames = len(files)
    landmark_tensor = np.zeros((num_frames, 478, 3), dtype=np.float32)

    with mp_face_mesh.FaceMesh(
            static_image_mode=True,
            max_num_faces=1,
            refine_landmarks=True,
            min_detection_confidence=0.5) as face_mesh:
        
        for i, file in enumerate(files):
            print(f"Elaborazione di {file}")
            image = cv2.imread(f"data/frames/{file}")
            if image is None:
                raise ValueError(f"Immagine {file} non trovata o non caricata correttamente.")

            # Rileva il volto
            with mp_face_detection.FaceDetection(min_detection_confidence=0.5) as face_detection:
                detection_results = face_detection.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
                if detection_results.detections:
                    bboxC = detection_results.detections[0].location_data.relative_bounding_box
                    ih, iw, _ = image.shape
                    x, y, w, h = int(bboxC.xmin * iw), int(bboxC.ymin * ih), int(bboxC.width * iw), int(bboxC.height * ih)
                    margin_w, margin_h = int(0.25 * w), int(0.25 * h)
                    x, y = max(0, x - margin_w), max(0, y - margin_h)
                    w, h = min(iw, w + 2 * margin_w), min(ih, h + 2 * margin_h)
                    face_crop = image[y:y+h, x:x+w]
                    image = cv2.resize(face_crop, (256, 256))
            
            # Ottieni i landmark
            results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            if results.multi_face_landmarks:
                for idx, landmark in enumerate(results.multi_face_landmarks[0].landmark):
                    landmark_tensor[i, idx] = [landmark.x, landmark.y, landmark.z]
    
    # Usa i canali rosso, verde e blu
    rgb_tensor = np.zeros_like(landmark_tensor, dtype=np.uint8)
    rgb_tensor[:, :, 0] = landmark_tensor[:, :, 0] * 255  # Rosso
    rgb_tensor[:, :, 1] = landmark_tensor[:, :, 1] * 255  # Verde
    rgb_tensor[:, :, 2] = landmark_tensor[:, :, 2] * 255  # Blu
    
    # Plot del tensore come immagine
    plt.figure(figsize=(10, 5))
    plt.imshow(rgb_tensor.reshape(num_frames, 478, 3))  # Visualizza l'immagine
    plt.axis('off')
    plt.show()
    
    #save image
    plt.imsave('data/landmarks.jpg', rgb_tensor.reshape(num_frames, 478, 3))

process_frames_and_plot()


Name tensor

In [ ]:
import pandas as pd
import os

def name_tensor(folder):
    # Take the name after the first underscore
    id_video = os.path.basename(folder).split('_', 1)[1]
    
    # Navigate to the folder before, e.g., frames → user folder
    id_user = folder.split('/')[-3]
    
    # Read the CSV file
    csv_path = f'data/users/{id_user}/{id_user}.csv'
    if not os.path.exists(csv_path):
        print(f"⚠️ CSV file not found: {csv_path}")
        return None  # Scarta il tensore
    
    df = pd.read_csv(csv_path)
    df.columns = df.columns.str.strip()  # Remove spaces in column names
    
    # Match id_video
    row = df.loc[df['Stimulus'].str.strip() == id_video.strip()]
    if row.empty:
        print(f"❌ ID video '{id_video}' non trovato nel CSV di {id_user}, tensore scartato.")
        return None  # Scarta il tensore
    
    arousal = row['Arousal'].values[0]
    valence = row['Valence'].values[0]
    likeability = row['Likability'].values[0]
    rewatch = row['Rewatch'].values[0]
    
    return f'{id_user}_{id_video}_{arousal}_{valence}_{likeability}_{rewatch}.npy'


Create tensors

In [ ]:
import os
import cv2
import numpy as np
import mediapipe as mp

# Imposta la libreria MediaPipe
mp_face_mesh = mp.solutions.face_mesh
mp_face_detection = mp.solutions.face_detection

# Funzione per estrarre i landmark per ogni frame
def extract_landmarks_from_frames(video_folder):
    # Ottieni tutti i frame ordinati
    files = sorted(os.listdir(video_folder))
    num_frames = len(files)
    landmark_tensor = np.zeros((num_frames, 478, 3), dtype=np.float32)

    with mp_face_mesh.FaceMesh(
            static_image_mode=True,
            max_num_faces=1,
            refine_landmarks=True,
            min_detection_confidence=0.5) as face_mesh:
        
        for i, file in enumerate(files):
            image = cv2.imread(f"{video_folder}/{file}")
            if image is None:
                raise ValueError(f"Immagine {file} non trovata o non caricata correttamente.")

            # Rileva il volto con Face Detection
            with mp_face_detection.FaceDetection(min_detection_confidence=0.5) as face_detection:
                detection_results = face_detection.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
                if detection_results.detections:
                    bboxC = detection_results.detections[0].location_data.relative_bounding_box
                    ih, iw, _ = image.shape
                    x, y, w, h = int(bboxC.xmin * iw), int(bboxC.ymin * ih), int(bboxC.width * iw), int(bboxC.height * ih)
                    margin_w, margin_h = int(0.25 * w), int(0.25 * h)
                    x, y = max(0, x - margin_w), max(0, y - margin_h)
                    w, h = min(iw, w + 2 * margin_w), min(ih, h + 2 * margin_h)
                    face_crop = image[y:y+h, x:x+w]
                    image = cv2.resize(face_crop, (256, 256))

            # Ottieni i landmark usando FaceMesh
            results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            if results.multi_face_landmarks:
                for landmarks in results.multi_face_landmarks:
                    # Salva i landmarks in landmark_tensor
                    for j, landmark in enumerate(landmarks.landmark):
                        landmark_tensor[i, j] = [landmark.x, landmark.y, landmark.z]

    return landmark_tensor


def process_videos():
    base_path = 'data/users'

    for user in os.listdir(base_path):
        user_path = os.path.join(base_path, user)
        frames_path = os.path.join(user_path, 'frames')

        if not os.path.isdir(frames_path):
            continue

        for video_folder in os.listdir(frames_path):
            video_folder_path = os.path.join(frames_path, video_folder)

            if os.path.isdir(video_folder_path):
                print(f"🎬 Elaborazione del video {video_folder} per l'utente {user}")
                
                landmark_tensor = extract_landmarks_from_frames(video_folder_path)
                
                tensor_name = name_tensor(video_folder_path.replace('\\', '/'))
                if tensor_name is None:
                    continue  # Se il tensore è da scartare, passa al prossimo video
                
                np.save(f"data/users/{user}/tensors/{tensor_name}", landmark_tensor)


process_videos()


Delete tensors

In [ ]:
import os

def delete_all_tensors():
    base_path = 'data/users'

    for root, _, files in os.walk(base_path):  # Scansiona tutte le cartelle e sottocartelle
        for file in files:
            if file.endswith('.npy'):  # Controlla se il file è un tensore
                file_path = os.path.join(root, file)
                os.remove(file_path)
                print(f"Eliminato: {file_path}")

# Esegui la funzione
delete_all_tensors()
